<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [11]:
import numpy as np
import pandas as pd
import gensim
import sys, re, os

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim
import zipfile 

import spacy
nlp = spacy.load('en_core_web_lg')



In [12]:
data_path = os.path.join(os.getcwd(), '../module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip')

archive = zipfile.ZipFile(data_path)
files = archive.namelist()

archive.extract(files[0])
df = pd.read_csv(files[0])
os.remove(files[0])

In [13]:
def tokenize(text):
    return [token for token in simple_preprocess(text, deacc=True) if token not in STOPWORDS]

In [16]:
df.head(5)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,tokens
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...","[order, item, bad, quality, missing, backup, s..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...","[bulk, expensive, way, products, like]"
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...","[duracell, price, happy]"
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht...","[work, brand, batteries, better, price]"
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht...","[batteries, long, lasting, price, great]"


In [14]:
df['tokens'] = df['reviews.text'].apply(lambda x: tokenize(x))
df['tokens'].head()

0    [order, item, bad, quality, missing, backup, s...
1               [bulk, expensive, way, products, like]
2                             [duracell, price, happy]
3              [work, brand, batteries, better, price]
4             [batteries, long, lasting, price, great]
Name: tokens, dtype: object

In [17]:
id2word = corpora.Dictionary(df['tokens'])

In [18]:
sys.getsizeof(id2word)

56

In [19]:
len(id2word.keys())

9621

In [20]:
id2word.filter_extremes(no_below=5, no_above=0.95)

In [21]:
len(id2word.keys())

3582

In [23]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [26]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [27]:
lda = LdaMulticore(corpus=corpus,
                  id2word=id2word,
                  random_state=42,
                  num_topics=10,
                  passes=10,
                  workers=4)


In [28]:
lda.print_topics()

[(0,
  '0.043*"amazon" + 0.026*"tablet" + 0.024*"apps" + 0.017*"good" + 0.016*"like" + 0.016*"store" + 0.014*"google" + 0.013*"best" + 0.013*"play" + 0.012*"prime"'),
 (1,
  '0.083*"kindle" + 0.038*"books" + 0.034*"read" + 0.027*"reading" + 0.026*"use" + 0.021*"bought" + 0.018*"love" + 0.018*"easy" + 0.016*"wife" + 0.016*"reader"'),
 (2,
  '0.081*"tablet" + 0.027*"great" + 0.022*"games" + 0.020*"kids" + 0.016*"use" + 0.014*"apps" + 0.014*"movies" + 0.014*"love" + 0.014*"bought" + 0.010*"memory"'),
 (3,
  '0.179*"good" + 0.107*"price" + 0.097*"great" + 0.041*"value" + 0.039*"batteries" + 0.030*"quality" + 0.030*"product" + 0.025*"deal" + 0.018*"far" + 0.017*"long"'),
 (4,
  '0.106*"batteries" + 0.040*"work" + 0.025*"long" + 0.020*"great" + 0.020*"brand" + 0.018*"amazon" + 0.016*"price" + 0.015*"buy" + 0.013*"brands" + 0.012*"battery"'),
 (5,
  '0.045*"loves" + 0.038*"old" + 0.035*"year" + 0.034*"tablet" + 0.032*"kids" + 0.031*"bought" + 0.027*"love" + 0.023*"games" + 0.021*"great" + 0.0

In [29]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [30]:
topics = [' '.join(t[0:5]) for t in words]

In [31]:
for id, t in enumerate(topics):
    print(f"----- topic {id} -----")
    print(t, end="\n\n")



----- topic 0 -----
amazon tablet apps good like

----- topic 1 -----
kindle books read reading use

----- topic 2 -----
tablet great games kids use

----- topic 3 -----
good price great value batteries

----- topic 4 -----
batteries work long great brand

----- topic 5 -----
loves old year tablet kids

----- topic 6 -----
great tablet easy price use

----- topic 7 -----
battery life tablet great long

----- topic 8 -----
batteries bought charge ve battery

----- topic 9 -----
screen kindle device like light



In [32]:
pyLDAvis.enable_notebook()

In [33]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/Users/ECF/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.224407 -0.052298       1        1  17.221325
5     -0.205291  0.076231       2        1  13.793800
0     -0.014620  0.039187       3        1  11.604545
2     -0.145735  0.044286       4        1   9.724983
6     -0.038960  0.180010       5        1   9.289110
1     -0.199649 -0.064448       6        1   8.458624
8      0.152666 -0.213887       7        1   8.179394
9     -0.086550 -0.162829       8        1   8.078242
3      0.268155  0.180214       9        1   8.061398
7      0.045577 -0.026465      10        1   5.588575, topic_info=     Category         Freq       Term        Total  loglift  logprob
34    Default  5913.000000       good  5913.000000  30.0000  30.0000
18    Default  7219.000000  batteries  7219.000000  29.0000  29.0000
17    Default  4998.000000      price  4998.000000  28.0000  28.0000
21    Default  8887.000000      great  8887.000000  27.0000  27.0000
2433  Default  6808.000000     tablet  6808.000000  26.0000  26.0000
2076  Default  3049.000000     kindle  3049.000000  25.0000  25.0000
1857  Default  2141.000000      loves  2141.000000  24.0000  24.0000
2     Default  2052.000000    battery  2052.000000  23.0000  23.0000
755   Default  2906.000000       easy  2906.000000  22.0000  22.0000
387   Default  1063.000000      value  1063.000000  21.0000  21.0000
9     Default  2423.000000       work  2423.000000  20.0000  20.0000
521   Default  2114.000000        old  2114.000000  19.0000  19.0000
105   Default  1751.000000       year  1751.000000  18.0000  18.0000
2087  Default  1504.000000      books  1504.000000  17.0000  17.0000
652   Default  1685.000000      games  1685.000000  16.0000  16.0000
1679  Default  1716.000000     screen  1716.000000  15.0000  15.0000
23    Default  2382.000000       long  2382.000000  14.0000  14.0000
192   Default  2440.000000       kids  2440.000000  13.0000  13.0000
786   Default  1222.000000       read  1222.000000  12.0000  12.0000
59    Default  1006.000000       life  1006.000000  11.0000  11.0000
1548  Default  1864.000000       apps  1864.000000  10.0000  10.0000
7     Default  1339.000000    quality  1339.000000   9.0000   9.0000
165   Default  1997.000000    product  1997.000000   8.0000   8.0000
28    Default  3155.000000     bought  3155.000000   7.0000   7.0000
90    Default  3418.000000       love  3418.000000   6.0000   6.0000
1268  Default   930.000000   daughter   930.000000   5.0000   5.0000
20    Default  1245.000000      brand  1245.000000   4.0000   4.0000
946   Default  1206.000000    reading  1206.000000   3.0000   3.0000
432   Default   847.000000       deal   847.000000   2.0000   2.0000
179   Default  1145.000000       nice  1145.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
1034  Topic10    37.928001     single    47.696404   2.6553  -6.1163
2     Topic10   941.233765    battery  2052.220703   2.1050  -2.9047
3     Topic10   212.899475       item   378.255005   2.3097  -4.3911
934   Topic10    61.184490    special    90.434036   2.4937  -5.6380
426   Topic10   112.693970      lasts   198.567871   2.3180  -5.0273
1007  Topic10    84.601540    charged   139.590973   2.3837  -5.3140
822   Topic10   106.278404      issue   194.292618   2.2811  -5.0859
1072  Topic10    58.969280   customer   101.768356   2.3388  -5.6749
178   Topic10    36.430935    knowing    52.386707   2.5212  -6.1565
1017  Topic10    88.240540      takes   212.393539   2.0061  -5.2719
798   Topic10    79.259682    service   182.224823   2.0519  -5.3792
731   Topic10    51.730606      cable    97.112030   2.2546  -5.8059
217   Topic10   152.639175     charge   610.236328   1.4987  -4.7239
1560  Topic10   100.439728      phone   312.307068   1.7500  -5.1424
244   Topic10   156.868103       fast   700.475830   1.3881  -4.6965
23    Topic10   294.730377       long  2382.848145   0.7945  -4.06

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling